https://www.medicinenet.com/

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

# params = {
#     'ie': 'UTF-8',
#     'wd': 'Python'
# }


In [8]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
    res = requests.get('https://www.medicinenet.com/', headers=headers_pc, proxies= randomproxy,)
    # 将cookieJar数据转化为字典，s
    cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

In [4]:
def html_download(tag, headers_pc, randomproxy, cookie_dict, item):
    # pbar.set_description(item['term'])
    try:
        url = item['url']
        term = item['term'].strip().replace('/', ' ').replace('?', '').replace('\\', ' ').replace('*', ' ').replace('\"', ' ')
        if os.path.exists(f"D:/Intern_Project/10_MedicineNet/{tag}_htmls/{term}.html"):
#         if os.path.exists(f"D:/Intern_Project/5_everydayhealth/health_html/{newterm}.html"):
            pbar.update(1)    
            return None 
#         time.sleep(random.randint(0, 5))
        r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        html = r.text
        
        with open(f"D:/Intern_Project/10_MedicineNet/{tag}_htmls/{term}.html", 'w+', encoding='utf-8') as f:
#         with open(f"D:/Intern_Project/5_everydayhealth/health_html/{newterm}.html", 'w+', encoding='utf-8') as f:
            f.write(html)
    
        pbar.update(1)
    
    except Exception as e:
        print(e)
        time.sleep(5)
        headers_pc, randomproxy, cookie_dict = changeparams()
        html_download(tag, headers_pc, randomproxy, cookie_dict, item)
    return None

In [5]:
def task_monitor(task_list, mt_pool, results):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
#                 results.extend(result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [ ]:
# 下载三种article_html
tags = ['diseases_and_conditions', 'medications', 'health_and_living']

for tag in tags:
    print(tag)
    with open(f'D:/Intern_Project/10_MedicineNet/{tag}术语链接列表.json', 'r', encoding='utf-8') as f:
            url_list = json.load(f)
    
    pbar = tqdm(total=len(url_list))
    
    all_craw_tasks = []
    results = []
    headers_pc, randomproxy, cookie_dict = changeparams()
    with ThreadPoolExecutor(max_workers=10) as mt_pool:
        for item in url_list:
            tmp_task = mt_pool.submit(html_download, tag, headers_pc, randomproxy, cookie_dict, item)
            all_craw_tasks.append(tmp_task)
#             time.sleep(0.5)
        # 监测下载任务进展
        task_monitor(all_craw_tasks, mt_pool, results)
    time.sleep(10)

diseases_and_conditions


  1%|▋                                                                             | 89/9512 [00:40<1:10:28,  2.23it/s]

HTTPSConnectionPool(host='www.medicinenet.com', port=443): Read timed out. (read timeout=10)


  2%|█▍                                                                           | 170/9512 [01:14<1:44:26,  1.49it/s]

HTTPSConnectionPool(host='www.medicinenet.com', port=443): Read timed out. (read timeout=10)


...

Invalid URL '/script/main/art.asp?articlekey=141757': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141757?
Invalid URL '/script/main/art.asp?articlekey=141534': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141534?
Invalid URL '/script/main/art.asp?articlekey=193133': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=193133?
Invalid URL '/script/main/art.asp?articlekey=46030': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=46030?
Invalid URL '/script/main/art.asp?articlekey=192412': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=192412?
Invalid URL '/script/main/art.asp?articlekey=225858': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=225858?
Invalid URL '/script/main/art.asp?articlekey=142022': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=142022?
Invalid URL '/script/m

Invalid URL '/script/main/art.asp?articlekey=142410': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=142410?
Invalid URL '/script/main/art.asp?articlekey=141757': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141757?
Invalid URL '/script/main/art.asp?articlekey=141534': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141534?
Invalid URL '/script/main/art.asp?articlekey=193133': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=193133?
Invalid URL '/script/main/art.asp?articlekey=192412': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=192412?
Invalid URL '/script/main/art.asp?articlekey=46030': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=46030?
Invalid URL '/script/main/art.asp?articlekey=142022': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=142022?Invalid URL '/script/ma

Invalid URL '/script/main/art.asp?articlekey=142410': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=142410?
Invalid URL '/script/main/art.asp?articlekey=141534': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141534?Invalid URL '/script/main/art.asp?articlekey=141757': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141757?

Invalid URL '/script/main/art.asp?articlekey=141637': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141637?
Invalid URL '/script/main/art.asp?articlekey=193133': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=193133?
Invalid URL '/script/main/art.asp?articlekey=46030': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=46030?
Invalid URL '/script/main/art.asp?articlekey=225858': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=225858?
Invalid URL '/script/m

Invalid URL '/script/main/art.asp?articlekey=193133': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=193133?
Invalid URL '/script/main/art.asp?articlekey=141637': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141637?
Invalid URL '/script/main/art.asp?articlekey=141534': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141534?
Invalid URL '/script/main/art.asp?articlekey=141757': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141757?
Invalid URL '/script/main/art.asp?articlekey=46030': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=46030?
Invalid URL '/script/main/art.asp?articlekey=141486': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141486?
Invalid URL '/script/main/art.asp?articlekey=142022': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=142022?
Invalid URL '/script/m

Invalid URL '/script/main/art.asp?articlekey=141637': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141637?
Invalid URL '/script/main/art.asp?articlekey=142410': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=142410?
Invalid URL '/script/main/art.asp?articlekey=141534': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141534?Invalid URL '/script/main/art.asp?articlekey=193133': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=193133?

Invalid URL '/script/main/art.asp?articlekey=141757': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=141757?
Invalid URL '/script/main/art.asp?articlekey=46030': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=46030?
Invalid URL '/script/main/art.asp?articlekey=192412': No scheme supplied. Perhaps you meant https:///script/main/art.asp?articlekey=192412?Invalid URL '/script/ma

In [6]:
# health_and_living的情况不一样，得单独搞一下
def sp_html_download(tag, headers_pc, randomproxy, cookie_dict, item):
    # pbar.set_description(item['term'])
    try:
        url = 'https://www.medicinenet.com' + item['url']
        term = item['term'].strip().replace('/', ' ').replace('?', '').replace('\\', ' ').replace('*', ' ').replace('\"', ' ')
        if os.path.exists(f"D:/Intern_Project/10_MedicineNet/{tag}_htmls/{term}.html"):
#         if os.path.exists(f"D:/Intern_Project/5_everydayhealth/health_html/{newterm}.html"):
            pbar.update(1)    
            return None 
#         time.sleep(random.randint(0, 5))
        r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        html = r.text
        
        with open(f"D:/Intern_Project/10_MedicineNet/{tag}_htmls/{term}.html", 'w+', encoding='utf-8') as f:
#         with open(f"D:/Intern_Project/5_everydayhealth/health_html/{newterm}.html", 'w+', encoding='utf-8') as f:
            f.write(html)
    
        pbar.update(1)
    
    except Exception as e:
        print(e)
        time.sleep(5)
        headers_pc, randomproxy, cookie_dict = changeparams()
        html_download(tag, headers_pc, randomproxy, cookie_dict, item)
    return None

In [10]:
# health_and_living的情况不一样
tag = 'health_and_living'
with open(f'D:/Intern_Project/10_MedicineNet/{tag}术语链接列表.json', 'r', encoding='utf-8') as f:
    url_list = json.load(f)
    
pbar = tqdm(total=len(url_list))

all_craw_tasks = []
results = []
headers_pc, randomproxy, cookie_dict = changeparams()
with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for item in url_list:
        tmp_task = mt_pool.submit(sp_html_download, tag, headers_pc, randomproxy, cookie_dict, item)
        all_craw_tasks.append(tmp_task)
#             time.sleep(0.5)
    # 监测下载任务进展
    task_monitor(all_craw_tasks, mt_pool, results)

 73%|█████████████████████████████████████████████████████████████                       | 677/931 [04:14<01:10,  3.59it/s]

HTTPSConnectionPool(host='www.medicinenet.comhttps', port=443): Max retries exceeded with url: //www.medicinenet.com/protein_quiz/faq.htm (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D6D7382790>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


 85%|███████████████████████████████████████████████████████████████████████▌            | 793/931 [05:00<00:32,  4.30it/s]

HTTPSConnectionPool(host='www.medicinenet.comhttps', port=443): Max retries exceeded with url: //www.medicinenet.com/summer_food_safety_quiz/faq.htm (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D6D76F39D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


100%|████████████████████████████████████████████████████████████████████████████████████| 931/931 [05:52<00:00,  2.19it/s]